In [38]:
from bs4 import BeautifulSoup as bs
import re
from dateutil.parser import parse
# from datetime import datetime
import pandas as pd
import json

In [6]:
def parse_html(path): 
    with open(path) as f:
        # read file
        content = f.read()
        # parse with bs
        soup = bs(content, 'lxml')

    return soup

In [7]:
watch_history = parse_html('../Rådata/Takeout/YouTube and YouTube Music/history/watch-history.html')

In [8]:
def is_a_video(tag): 
    '''
    Filter to remove ads and videos that still ex
    takes a block of html code and returns a boolean with whether this is an ad or not
    '''
    not_an_ad = not(bool(re.search('From Google Ads', tag.get_text())))
    not_deleted = len(tag.find_all('a')) > 2
    return (not_an_ad & not_deleted)
       
 

In [9]:
def parse_watch_history(soup):
    '''takes a parsed tree of watch history (wh) from beautiful soup and returns a simple df with the relevant data'''
    # hver yderste blok har klassen 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp' - Her henter vi dem alle sammen
    blocks = soup.find_all(class_ = 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')
    blocks_cleaned = [block for block in blocks if is_a_video(block)]
    print(f'{len(blocks)} entries; {len(blocks_cleaned)} after cleaning. {len(blocks) - len(blocks_cleaned)} ads and deleted videos removed!')
    return blocks_cleaned

watch_history_blocks = parse_watch_history(watch_history)

30345 entries; 28332 after cleaning. 2013 ads and deleted videos removed!


In [10]:
# Finding a video and an add for testing! 
video = watch_history_blocks[0]
ad =  watch_history_blocks[5]

# To create the dataset! 

Det store problem her er at få datoen til at blive hentet rigtigt fra teksten solvom der kan være overlap med titlen af kanalen (hvis den slutter på et tal) eller hvis den har en mærkelig sætning der siger hvornår den er set på - det kan forekomme i nogle tilfælde. 

Den foreløbige løsning er kombinationen af f og r strenge - her kan variable puttes ind i en regex. Det er vigtigt at huske på at kvantifiers i regex '{}' så skal have et dobbelt sæt klammer rund om sig!! 

In [11]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 
watched_at_videos = [video for video in watch_history_blocks if re.search(r'Watched at \d\d:\d\d', video.text)]

In [131]:
watched_at_video = watched_at_videos[0]
watched_at_video.text
match = re.search(r'Watched at \d\d:\d\d', watched_at_video.text)
match.group()

'Watched at 22:16'

In [132]:
# All of this is to find out how to do the cleansing properly - it should work now but it is here for posterity in case it needs fixing! 

# match = re.search(watch_history_blocks[0].text, r'\\n\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*')
# print(match)
# watch_history_blocks[0].find(class_ = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1').text
block = watched_at_video
print(block.text)
links = block.find_all('a')

channel_title = re.escape(links[1].text)
watched_at = re.search(r'Watched at \d\d:\d\d', block.text) # Watched at 22:16

if watched_at:
   search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
else:
    search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

print(search_string)

match = re.search(search_string, block.text)

print(search_string, match)
match[1]

YouTubeWatched YouTube Music Ableton Walkthroughs: Part 1Red Means RecordingWatched at 22:169 Nov 2019, 23:05:11 CESTProducts: YouTubeWhy is this here? This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  here.
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*)
Red\ Means\ RecordingWatched at 22:16(\d?\d \w{3,4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*) <re.Match object; span=(57, 117), match='Red Means RecordingWatched at 22:169 Nov 2019, 23>


'9 Nov 2019, 23:05:11 CEST'

In [133]:
video_titles = []
video_links = []
channel_titles = []
channel_links = []
date_watched = []

# i = 0 # debugging! 
for block in watch_history_blocks:
    links = block.find_all('a')
    
    video_titles.append(links[0].text)
    video_links.append(links[0]['href'])
    channel_title = links[1].text # get channel title for date search! 
    channel_titles.append(channel_title)
    channel_links.append(links[1]['href'])

    # Useful for debugging
    # i += 1
    # print(f'{len(links)} links; entry {i} with title {links[0].text}')

    # find out if text has the weird watched at text so we can avoid it! 
    watched_at = re.search(r'Watched at \d\d:\d\d', block.text) 
    channel_title = re.escape(channel_title) # escape so we avoid problems with channel titles full of weird characters! 

    if watched_at:
        search_string = fr'{channel_title}{watched_at.group()}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)'  
    else:
        search_string = fr'{channel_title}(\d?\d \w{{3,4}} \d{{4}}, \d{{2}}:\d{{2}}:\d{{2}}[^P]*)' 

    date_string = re.search(search_string, block.text)

    if watched_at:
        date_watched.append(parse(date_string.group(1)))
    else:
        date_watched.append(parse(date_string.group(1)))


In [146]:
# making sure that we only get the ids! 
for video in video_links:
    match = re.search(r'https://www.youtube.com/watch\?v=(.*)', video)
    if not match:
        print(video)

In [22]:
type(date_watched[0])

datetime.datetime

In [17]:
date_string = re.search(r'\d?\d \w{3} \d{4}, \d{2}:\d{2}:\d{2}[^P]*', video.text)


In [18]:
date = parse(date_string[0])
date_string[0]

'9 Aug 2022, 13:27:58 CEST'

In [31]:
print(date.isoformat())
print(pd.Timestamp(date))

2022-08-09T13:27:58+02:00
2022-08-09 13:27:58+02:00


In [147]:
# dataframe = pd.DataFrame({
#     'video_title' : video_titles,
#     'video_link' : video_links,
#     'channel_title' : channel_titles,
#     'channel_link' : channel_links,
#     'date_watched' : date_watched
# })
dataframe = pd.read_csv('../Renset data/watch_history_df.csv', index_col=0, parse_dates=['date_watched'])

In [148]:
dataframe.rename(columns={'video_link' : 'video_id'}, inplace = True)

In [149]:
dataframe.loc[0, 'date_watched'].isoformat()
# dataframe.shape

'2022-08-09T13:27:58+02:00'

# Getting more data from the API

In [109]:
# new imports! 
import googleapiclient.discovery
import google_auth_oauthlib.flow

In [51]:
# points used for downloading all my data
#  
# points_pr_day = 10000
# n_videos = 28332
# caption_points = 200 * n_videos
# video_info = 0 * n_videos

# points = caption_points + video_info
# print(f'points used for getting all captions: {points:.1E}, \n\
# points in a day: {points_pr_day:.1E}')


points used for getting all captions: 5.7E+06, 
points in a day: 1.0E+04


In [150]:
with open('../credentials.json') as f:
    credentials = json.load(f)

In [153]:
test_data = dataframe.head(1)
ids = test_data.video_id

In [152]:
def get_caption(video_id):
    '''
    Takes a video ID and retrieves the caption from YouTubes API
    '''
    pass
    
api_service_name = "youtube"
api_version = "v3"

ids = test_data['video_id']
client_secrets_file = '../credentials.json' 
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

# API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)

credentials = flow.run_console()

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=859790329153-3ncuv1l9tuhpb0nocncthbhbiv62r7jm.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=aXgvXp0qgej2ch9AMeWy93feR0bD7i&prompt=consent&access_type=offline


In [154]:
ids

0    oPpzT6JKGk0
Name: video_id, dtype: object

In [156]:
request = youtube.captions().list(
        part="snippet",
        videoId=ids[0]
    )

response = request.execute()

print(response)

{'kind': 'youtube#captionListResponse', 'etag': 'POYMJm1npY1EZRySg8qwTUvFyso', 'items': [{'kind': 'youtube#caption', 'etag': 'NB05FRBEu0z6askKT8n_8zmnMR0', 'id': 'AUieDaYGRdTIARzPgskcXiH3ZmPcjGvUZrXIFRA7az4Z4QfyJ3Q', 'snippet': {'videoId': 'oPpzT6JKGk0', 'lastUpdated': '2022-07-30T07:16:09.575042Z', 'trackKind': 'asr', 'language': 'en', 'name': '', 'audioTrackType': 'unknown', 'isCC': False, 'isLarge': False, 'isEasyReader': False, 'isDraft': False, 'isAutoSynced': False, 'status': 'serving'}}, {'kind': 'youtube#caption', 'etag': 'tpqgwaRR0pf0vTcw91dQt9DXEII', 'id': 'AUieDaZKDswN2qMsRgAHATSSIF6rT84FwWmad2grobN-', 'snippet': {'videoId': 'oPpzT6JKGk0', 'lastUpdated': '2022-07-29T14:41:49.876101Z', 'trackKind': 'standard', 'language': 'en', 'name': '', 'audioTrackType': 'unknown', 'isCC': False, 'isLarge': False, 'isEasyReader': False, 'isDraft': False, 'isAutoSynced': False, 'status': 'serving'}}]}


In [119]:
request = youtube.captions().download(
    id = 'oPpzT6JKGk0'
)

response = request.execute()

print(response)

HttpError: <HttpError 404 when requesting https://youtube.googleapis.com/youtube/v3/captions/watch%3Fv%3DoPpzT6JKGk0? returned "The caption track could not be found. Check the value of the request's <code>id</code> parameter to ensure that it is correct.". Details: "[{'message': "The caption track could not be found. Check the value of the request's <code>id</code> parameter to ensure that it is correct.", 'domain': 'youtube.caption', 'reason': 'captionNotFound', 'location': 'id', 'locationType': 'parameter'}]">

In [116]:
ids[0]

'watch?v=oPpzT6JKGk0'